In [7]:
import json
from collections import Counter
import pandas as pd

In [3]:
DATA_DIR = '/Users/ik/Data/spook/'

In [4]:
train_data = pd.read_csv(DATA_DIR + 'train.csv')

In [9]:
Counter(train_data.author)

Counter({'EAP': 7900, 'HPL': 5635, 'MWS': 6044})

### Edgar Allan Poe
* The Cask of Amontillado - http://www.gutenberg.org/ebooks/1063.txt.utf-8

* The Fall of the House of Usher - http://www.gutenberg.org/ebooks/932.txt.utf-8

* The Masque of the Red Death - http://www.gutenberg.org/ebooks/1064.txt.utf-8

* The Raven - http://www.gutenberg.org/ebooks/17192.txt.utf-8

* The Works of Edgar Allan Poe — Volume 1 - http://www.gutenberg.org/files/2147/2147-0.txt

* The Works of Edgar Allan Poe — Volume 2 - http://www.gutenberg.org/files/2148/2148-0.txt